In [2]:
from zipfile import ZipFile

# opens the chess boards.zip file into the chessBoards object
zipFilePath = './archive.zip'
chessBoards = ZipFile(zipFilePath, 'r')

#datasetTest = []
#datasetTrain = []
test = []
train = []
newpath = True

# extract the file names from the train and test folders in the zip archive
for file in chessBoards.namelist():
    # fills the file names from within the dataset subfolder
    """if file[:13] == 'dataset/test/':
        datasetTest.append(file)
    if file[:14] == 'dataset/train/':
        datasetTrain.append(file)"""

    if file[:4] == 'test':
        test.append(file)
    if file[:5] == 'train':
        train.append(file)

    # find any new file paths not accounted for
    """if file[:13] != 'dataset/test/' and \
        file[:14] != 'dataset/train/' and \
        file[:4] != 'test'and \
        file[:5] != 'train' and \
        newpath:
        print("new path: ", file)
        newpath = False"""

print("Files in test: ", len(test))
print("Files in train: ", len(train))



Files in test:  20000
Files in train:  80000


In [32]:
import pickle
from PIL import Image
import io
import os

# Extracts the data from the zip file and saves it to a pickle file
#
# @param[in] zipFilePath - the path to the zip file
# @param[in] outputPickleFile - the path to the pickle file to save the data to
# @param[in] directory - the directory within the zip file to extract the data from
# @param[in] num_samples - the number of samples to extract from the zip file
def extractData(zipFilePath, outputPickleFile, directory='test/', num_samples=None):
    data = []

    with ZipFile(zipFilePath, 'r') as chess_boards:
        for file_name in chess_boards.namelist():
            if file_name.startswith(directory):
                # Extract FEN string from file name
                fen_string = file_name.split('/')[-1]

                # Extract image bytes and convert to PIL Image
                with chess_boards.open(file_name) as image_file:
                    image_bytes = image_file.read()
                    image = Image.open(io.BytesIO(image_bytes))

                    # Converts image to grayscale
                    image = image.convert('L')

                # Append FEN string and image to data
                data.append((fen_string, image))

                # Check if the desired number of samples is reached
                if num_samples is not None and len(data) >= num_samples:
                    break

    # Get the directory path and base filename
    output_dir, base_filename = os.path.split(outputPickleFile)
    
    # Include the number of samples in the base filename
    if num_samples is not None:
        base_filename = f"{os.path.splitext(base_filename)[0]}_{num_samples}.pickle"
    
    # Construct the full output pickle file path
    outputPickleFile = os.path.join(output_dir, base_filename)

    # Save data to pickle file
    with open(outputPickleFile, 'wb') as pickle_file:
        pickle.dump(data, pickle_file)

    print(f"Data extracted and saved to {outputPickleFile}")

# Entry point to start the process of pickling the data
def pickleData():
    zipFilePath = './archive.zip'
    outputPickleFile = './test_data.pickle'
    # extractData(zipFilePath, outputPickleFile, "test/")

    outputPickleFile = './train_data.pickle'
    # extractData(zipFilePath, outputPickleFile, "train/")

pickleData()


In [34]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Trains a model using the training data from the pickle file
#
# @param[in] trainingDataFile - the path to the pickle file containing the training data
# @return - the trained Random Forest model
def trainModel(trainingDataFile):
    # Load the training data from the pickle file
    with open(trainingDataFile, 'rb') as file:
        trainingData = pickle.load(file)

    # Extract the images and labels from the data
    X_train = [data[1].flatten() for data in trainingData]
    y_train = [data[0] for data in trainingData]

    # Train the Random Forest model
    model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=None)
    model.fit(X_train, y_train)

    return model

def evaluateModel(model, testDataFile):
    # Load the test data from the pickle file
    with open(testDataFile, 'rb') as file:
        testData = pickle.load(file)

    # Extract the images and labels from the data
    X_test = [data[1].flatten() for data in testData]
    y_test = [data[0] for data in testData]

    # Predict the labels using the model
    y_pred = model.predict(X_test)

    # Evaluate the model using accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model accuracy: {accuracy}")

# Entry point to train the model and evaluate it
def main():

    # Train
    print("Training model...")
    model = trainModel('./train_data.pickle')
    print("Model trained")

    # Evaluate
    print("Evaluating model...")
    evaluateModel(model, './test_data.pickle')

main()


Training model...


AttributeError: module 'pandas' has no attribute 'load'